# Tutorial 4: Classification using machine learning

#### Hello, and welcome to the final tutorial in this series!

This tutorial will show you how to:
 - Use different machine learning methods (logistic regression and (later) random forest) for classification
 - Use a pipeline to organize your training and testing.
 - Apply hyperparameter optimization to improve the classifiers
 - Compare those methods against each other and a baseline using jackknife
 
 Can't wait to get started! First as usual we download the necessary packages.

In [ ]:
# ___Cell no. 1___

## Python packages 
from sklearn.linear_model import LogisticRegression # a ML method
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

We also need the data stored in files `df` and `base_dict` which were created in Tutorial 3. 

In [ ]:
# ___Cell no. 2___

%store -r df
%store -r base_dict 

### Set up machine learning methods and their hyperparameters

Logistic regression is a leading machine learning (ML) method  (later on we will use random forest, which is another common method). We will compare the results from multiple classifiers. First we create aliases for the classifiers, and create a list to use when we apply the classifiers one by one.

For our initial trial we will use two variants of logistic regression:  with or without fitting an intercept.  Basically, fitting an intercept means that the line (or plane) that separates between the two classes can be any line, while not fitting an intercept means that the line of separation must pass through the origin)


In [ ]:
# ___Cell no. 3___

lr = LogisticRegression() # defining the model

## Define a list of models, We will use this when we loop through ML methods.
### Here we define two variants of logistic regression: one which fits an intercept, and one that does not.
### (basically, fitting an intercept means that the line of separation between the two classes can be any line,
### while not fitting an intercept means that the line of separation must pass through the origin)
models = [ [lr, 'lrFI'], [lr, 'lrNFI']]


ML methods have *hyperparameters* which must be defined before they can run. The hyperparameter values used can significantly affect the performance of the methods. hyperparameters specify the _process_ of fitting: they do not appear in the final data model, but they determine how the data model is computed. For this reason, *hyperparameter optimization* is used to find the best possible values for these hyperparameters. 
Basically, this is done using trial and error: each ML algorithm is trained with multiple combinations of hyperparameters, and the best combination is chosen (this is called *grid search*).
 
Each method has its own set of hyperparameters  In the following code block we set possible values for hyperparameters for logistic regression (LR). For example, we are using an L2 penalty. Another possible penalty is L1. To understand the difference see:

https://builtin.com/data-science/l2-regularization

Essentially, L2 penalty tends to equalize more the sizes of the regression coefficients, while L1 tends to favor more the coefficients for those features that are better predictors.

(It turns out that L1 doesn't work well on our dataset, so we will stick with L2.)

For more information on other hyperparameters for LR, as well as a more complete description of grid search check the following links:

For logistic regression hyperparameters:
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

For grid search:
- https://machinelearningmastery.com/hyperparameter-optimization-with-random-search-and-grid-search/

Other methods have other parameter sets. For example, the parameter sets for Random Forest (which we will use later) are here:

- https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html (RF)


In [ ]:
# ___Cell no. 4___

## logistic regression (LR) with intercept fitting
solvers = ['newton-cg', 'liblinear']
penalty = ['l2']
c_values = [16.0,4.0, 1.0, 0.25]
lrFI_par = dict(solver=solvers,penalty=penalty,C=c_values) # creating a dictionary that contains the hyperparamters

## @@@ Initialize parameters for LR without intercept and put in a dictionary named `lrNFI_par`. All parameters are the same, except set  the `fit_intercept` 
## @@@  option as False.  See the above documentation for LR.

## Define a list of parameter sets to go along with the list of models.
parameters = [lrFI_par, lrNFI_par]

To set up the training and testing sets, we need to specify inputs (feature sets), outputs (targets), and the training/testing split. We will apply each ML method to three different feature sets, and compare the performances. The code is very similar to the code for the baseline estimator (Mahalanobis distance). 

In [ ]:
# ___Cell no. 5___

y = df[['type']] # define the targets
features = [ ['TPC_H2O','FRAP_H2O'], ['TEAC_H2O','FRAP_H2O'], ['TPC_H2O', 'TEAC_H2O']] # define a list of features
splits = 0.4 # definplit (Feel free to replace with a better value)

#### Training

Just like the previous tutorial, the following code trains and evaluates multiple classifiers (3 feature sets with 2 ML models). For this purpose, we define a function `get_accuracy_ml` which produces the following outputs for each classifier:

* `tot_acc`    : total accuracy;
* `jack_trainSD` : jackknife estimate of standard deviation of accuracy due to training errors 
* `jack_testSD`  : jackknife estimate of standard deviation of accuracy due to testing error
* `jack_totSD`   : jackknife estimate of standard deviation of accuracy estimate.

The function `get_accuracy_ml` also requires another function `cv` that does _cross-validation_. A good reference for this is Jason Brownlee's "Machine Learning Mastery" blog:

https://machinelearningmastery.com/k-fold-cross-validation/

In [ ]:
# ___Cell no. 6___
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import *
from sklearn import metrics

def cv (m, p, xtrain, ytrain, xtest, ytest):
    inner_cv = StratifiedKFold(n_splits=3)
    # The following statement optimizes the model m among parameter options p, making use of the given training and testing set
    clf = GridSearchCV(m, p, scoring='accuracy', n_jobs=-1, cv=inner_cv, refit=True, verbose=0)
    clf.fit( xtrain,  ytrain)
    yPred = clf.predict(xtest)# This is the prediction derived from the optimized model on the testing set
    #@@@ Set 'result' equal to the accuracy calculated by comparing the predictions (yPred) and the true labels (ytrain)
    return result,clf
 

def get_accuracy_ml (m, p, xTrain, yTrain, xTest, yTest):
    accTot,clfTot = cv(m, p, xTrain, yTrain, xTest, yTest)
    
    # Compute jackknife estimates
    # Need number of estimates to define lists of results
    nxtr = len(yTrain)
    nxte=len(yTest)
    
    # Make space for lists of leave-out-one results
    # Use jackknife to estimate variation due to training error
    jackTrainArr = np.zeros(nxtr)
            
    for i in range(len(xTrain)):
        x_train = np.delete(np.array(xTrain), i, 0)
        y_train = np.delete(np.array(yTrain), i, 0)
        
        scoreTrain,clfTrain = cv(m, p, x_train, y_train, xTest, yTest)
        jackTrainArr[i] = scoreTrain
            
    # @@@ write a similar code to create jackTestArr
    
    # Compute SD's for error due to training and testing via jackknife formula
    # The total SD is computed using the variance addition formula
    jackTrainSD = # @@@ Complete this (similar to the code used for Mahalanobis)
    jackTestSD = # @@@ Complete this
    jackTotSD = np.sqrt(jackTrainSD**2 + jackTestSD**2)
    # Finished! return results
    return  accTot, jackTrainSD,jackTestSD,jackTotSD


Next we define an empty dictionary to hold our results.

In [ ]:
# ___Cell no. 7___

ml_dicts = {}

In [ ]:
# ___Cell no. 8___

# Loop through different ML models and hyper parameters combinations (use the same splits for all features)

for m, par in zip(models, parameters):
    
    X_train, X_test, y_train, y_test = train_test_split(df, y, test_size= splits, random_state=1, stratify = y, shuffle = True)
    
    # defining The main subkeys, which are the machine learning models
    key0 = str(m[1])
    print("Main key for the following iterations:")
    print(key0)
    
    # Define sub-dictionaries for each main key to hold results for different feature combinations
    ml_dicts[key0] = {} 
    
    # Loop through features
    for f in features:
        xtr =  X_train[f] 
        xte =  X_test[f]
        # Use `get_accuracy_ml` to obtain results for given classifier 
        results = get_accuracy_ml (m[0], par, np.array(xtr), np.array(y_train).flatten(), np.array(xte), np.array(y_test).flatten()) # to get the accuracies for the ml model
        
        key = str(splits)+","+str((f)) # Create keys for the each feature set in order to reference results
        # Create subdictionary for each feature combinations to hold results for that specific combo. 
        ml_dicts[key0][key] = {}

        # Put results into dictionary
        ml_dicts[key0][key]['tot_acc'] = results[0]
        # @@@ similarly, put the training, testing and total standard deviations in the dictionary 
        # @@@ (You may compare the code used for Mahalanobis)

Let's take a look at how the outputs are organized. We'll print some of the dictionary keys in the output data structure, which is called 'ml_dicts'.

In [ ]:
# ___Cell no. 9___

print("The main keys are: "+ str(ml_dicts.keys()) + " which stands for the main ML models\n")
print("The main subkeys for the first estimator are:")
print(str(ml_dicts['lr'].keys()) + " which stands for the main ML features\n")
print("The subkeys inside the first feature set:")
# @@@ Print out the subkeys for the first features set

**Exercise:** Display the same results for the second estimator

In [ ]:
#  ___ code here ____


---

#### Create arrays for results

As in Tutorial 3, we form arrays so that we can do a plot with error bars.

In [ ]:
# ___Cell no. 10___

arr_all = []
for m, d in zip (models, ml_dicts.keys()):
    acc_arr = [] 
    sd_arr = [] 

    # @@@ Write code to put the accuracies and total standard deviations for all models and feature sets 
    # @@@ in the arrays `acc_arr` and `sd_arr`, respectively. 

Just to double check we're OK, we print the results. (We will graph them in a minute.)

In [ ]:
# ___Cell no. 11___

print(arr_all) 

#### Graphing the results
By graphing the results and error bars for each feature set and method, we make it easy to compare accuracies across the different feature sets and ML methods.

In [ ]:
# ___Cell no. 12___

colors = ['blue', 'purple'] # colors for the different methods
plt.figure(figsize=(15, 7))
plt.title( "Accuracy  for different features with the SD", fontweight ='bold', fontsize =12)
plt.xlabel("Features", fontweight ='bold', fontsize =12)
plt.ylabel("Accuracy", fontweight ='bold', fontsize =12)

count = 0
n = 5

space = []
tickFeat = []

for result, model, color in zip(arr_all, models, colors):
    a = np.linspace(n*count, n*(1+count)-2,len(features)) # to get index on the x-axis
    space.extend(a)
    tickFeat.extend(result[0])
    plt.errorbar( a, result[1], result[2], fmt='o', label =model[1], color = color)
    count += 1

# @@@ Label the ticks on the x-axis with the feature set names
# @@@ Set reasonable y limits for your plot
# @@@ Add a legend that indicates the two types of LR model used.
# @@@ Show your plot using plt.show()


**Exercise:** Summarize your results. Which estimator appears to be most accurate? Which is the most stable (in terms of small error bars)? What about statistical significance? Does including intercept fitting provide any advantage?

### Displaying performance relative to the baseline

Do these classifiers really give us an advantage over a very simple statistical classifier that is based on Mahalanobis distance? We choose as our baseline the best Mahalanobis classifier that we found in the previous tutorial, and we plot the accuracy difference between our ML classifers and this baseline (with error bars).

First we create a list of relative accuracies and standard deviations for the different ML classifiers

In [ ]:
# ___Cell no. 13___

# List containing data for different classifiers
arr_diff_all = []

acc_base = base_dict["0.4"]["['TPC_H2O', 'TEAC_H2O']"][ 'tot_acc' ]
# @@@ Define `var_base` as the variance of the base estimator (square of standard deviation)

# Loop through models to complie all data
for m, m_key in zip (models, ml_dicts.keys()):
    acc_diff_arr = [] 
    sd_diff_arr = [] 
    for f_key in ml_dicts[m_key].keys():
        
        accDiff = ml_dicts[m_key][f_key][ 'tot_acc' ] -  acc_base
        acc_diff_arr.append(accDiff )
        # @@@ Obtain the standard deviation of the current model, call it 'sd'
        
        # @@@ Append to `sd_diff_arr` the standard deviation of the difference, which is the square root of the 
        # @@@ sum of base variance  + variance of current model  
        # @@@ (the variance of the current model is the square of sd)       
    arr_diff_all.append([ list(ml_dicts[m_key].keys()), acc_diff_arr, sd_diff_arr])  

The information is stored in a list that tells the train/test split and features; mean accuracy difference; and standard deviation of accuracy for the baseline and ML classifiers. 

In [ ]:
# ___Cell no. 14___

print(arr_diff_all)

Since the results are arranged in an orderly fashion, we can now graph them. Since we are measuring relative to the baseline, the baseline appears as a solid black line at 0.

In [ ]:
# ___Cell no. 15___

colors = ['purple', 'green'] # colors for the different methods
plt.figure(figsize=(15, 7))
plt.title( "Accuracy  difference vs. baseline for ML methods", fontweight ='bold', fontsize =12)
plt.xlabel("Features", fontweight ='bold', fontsize =12)
plt.ylabel("Accuracy  difference from baseline", fontweight ='bold', fontsize =12)
count = 0
n = 5
space = [] # will be used to label the x-axis
tickFeat = [] # will be used to label the x-axis

for result, model, color in zip(arr_diff_all, models, colors):
    a = np.linspace(n*count, n*(1+count)-2,len(features))
    space.extend(a)
    tickFeat.extend(result[0])
    plt.errorbar( a, result[1], result[2], fmt='o', label =model[1], color = color)
    count += 1
    
plt.plot(np.array(space), np.zeros(len(features)*len(models)), color = 'Black')        

# @@@ Label the ticks on the x-axis with the feature set names
# @@@ Set reasonable y limits for your plot
# @@@ Add a legend that indicates the two types of LR model used.
# @@@ Show your plot using plt.show()

**Exercise:** Summarize your results. Are there any estimators that appear to improve over the baseline? Is the improvement statistically significant?

### _The END_


<img src="pics/hap.jpg" width="300" height="200">
